# Kernel Principal Component Analysis

Muitos algoritmos de aprendizagem de maquinas assumem que os dados possam ser separados com uma funcao linear. O perceptron mesmo requer que os dados sejam perfeitamente separaveis para que o treinamento convirja. Outros algoritmos, como a regressao logistica, consideram a que a falta de separabilidade linear se deva a ruidos e anomalias. No entanto, quando lidamos com problemas nao lineares, os quais sao encontrados com bastante frequencia em aplicacoes reais, reducao de dados utilizando tecnicas lineares, como PCA e LDA, podem nao funcionar.

Nessa aula, veremos como a versao _kernelizada_ do PCA, ou KPCA, transforma dados nao linearmente separaveis em um sub-espaco de menor dimensao onde os dados podem ser linearmente separaveis. 



## Implementando o KPCA

No exemplo a seguir, utilizaremos os pacotes SciPy e NumPy para implementar nossa versao do KPCA:

In [13]:
from scipy.spatial.distance import pdist, squareform
from scipy import exp
from scipy.linalg import eigh
import numpy as np 
def rbf_kernel_pca(X, gamma, n_components):
    """
    Implementacao do PCA com kernel RBF (radial basis function).    
    Parameters
    ------------
    X: {NumPy ndarray}, shape = [n_examples, n_features]  
    gamma: float
        Parametro de ajuste do kernel RBF
    n_components: int
        Numero de componentes (features) a ser retornado    
    Retorna
    ------------
    X_pc: {NumPy ndarray}, shape = [n_examples, k_features]
        Dataset projetado no novo espaco  
    """
    # Computa a distancia euclidiana entre todos os pares de amostras
    # no espaco original.
    sq_dists = pdist(X, 'sqeuclidean')    
    # Converte as distancias entre os pares em uma matriz quadrada.
    mat_sq_dists = squareform(sq_dists)    
    # Computa a matriz de kernels simetricos.
    K = exp(-gamma * mat_sq_dists)    
    # centraliza a matriz de kernels.
    N = K.shape[0]
    one_n = np.ones((N,N)) / N
    K = K - one_n.dot(K) - K.dot(one_n) + one_n.dot(K).dot(one_n)    
    # Obtem os 'autopares' da matriz de kernels centralizada
    # a funcao scipy.linalg.eigh retorna esses valores em ordem ascendenter
    eigvals, eigvecs = eigh(K)
    eigvals, eigvecs = eigvals[::-1], eigvecs[:, ::-1]    
    # Pega os k primeiros autovetores na nova projecao
    X_pc = np.column_stack([eigvecs[:, i]
                           for i in range(n_components)])    
    return X_pc

Uma desvantagem de usar o PCA com kernel RBF para reducao de dimensionalidade e que precisamos especificar o parametro gamma a priori. Encontrar o valor apropriado requer experimentar diversos valores, e talvez usar alguma tecnica de otimizacao de parametros, como por exemplo grid search.

## Examplo — separando dados em formato de meia luas

Vamos aplicar nosso PCA com kernel RBF em um dataset nao linear. Comecamos criando um dataset bi-dimensional com 100 amostras representando duas formas de meia lua:

In [12]:
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt
X, y = make_moons(n_samples=100, random_state=123)
plt.scatter(X[y==0, 0], X[y==0, 1],
    color='red', marker='^', alpha=0.5)
plt.scatter(X[y==1, 0], X[y==1, 1],
    color='blue', marker='o', alpha=0.5)
plt.tight_layout()
plt.show()

Para uma melhor visualizacao, a meia lua de triangulos representa uma classe enquanto os circulos representam a outra classe.

Podemos ver claramente que as duas meia luas nao sao linearmente separaveis, e nosso objetivo e desdobrar as as meia luas via KPCA para que o dataset possa servir como uma entrada boa o suficiente para um classificador linear. Mas primeiramente, vamos ver como fica o dataset se fizermos a projecao utilizando o PCA tradicional:

In [11]:
from sklearn.decomposition import PCA
scikit_pca = PCA(n_components=2)
X_spca = scikit_pca.fit_transform(X)
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(7,3))
ax[0].scatter(X_spca[y==0, 0], X_spca[y==0, 1],
              color='red', marker='^', alpha=0.5)
ax[0].scatter(X_spca[y==1, 0], X_spca[y==1, 1],
              color='blue', marker='o', alpha=0.5)
ax[1].scatter(X_spca[y==0, 0], np.zeros((50,1))+0.02,
              color='red', marker='^', alpha=0.5)
ax[1].scatter(X_spca[y==1, 0], np.zeros((50,1))-0.02,
              color='blue', marker='o', alpha=0.5)
ax[0].set_xlabel('PC1')
ax[0].set_ylabel('PC2')
ax[1].set_ylim([-1, 1])
ax[1].set_yticks([])
ax[1].set_xlabel('PC1')
plt.tight_layout()
plt.show()

A projecao acima deixa bem claro que um classificador linear nao seria capaz de executar uma predicao adequada dos dados transformados utilizando o PCA tradicional.

Notem que ao plotarmos apenas os componentes principais (a direita), nos deslocamos os triangulos um pouquinho pra cima e os circulos um pouquinho pra baixo para conseguir visualizar melhor a sobreposicao das classes. Essa transformacao nao ajudaria um classificador linear a discriminar entre os circulos e triangulos. 

Agora vamos tentar com nosso PCA com kernel RBF, implementado acima:

In [10]:
X_kpca = rbf_kernel_pca(X, gamma=15, n_components=2)
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(7, 3))
ax[0].scatter(X_kpca[y==0, 0], X_kpca[y==0, 1],
              color='red', marker='^', alpha=0.5)
ax[0].scatter(X_kpca[y==1, 0], X_kpca[y==1, 1],
              color='blue', marker='o', alpha=0.5)
ax[1].scatter(X_kpca[y==0, 0], np.zeros((50,1))+0.02,
              color='red', marker='^', alpha=0.5)
ax[1].scatter(X_kpca[y==1, 0], np.zeros((50,1))-0.02,
              color='blue', marker='o', alpha=0.5)
ax[0].set_xlabel('PC1')
ax[0].set_ylabel('PC2')
ax[1].set_ylim([-1, 1])
ax[1].set_yticks([])
ax[1].set_xlabel('PC1')
plt.tight_layout()

Podemos observar que as duas classes (circulos e triangulos) sao linearmente separaveis, de modo que temos um dataset adequado para treinar um classificador linear.

Infelizmente, nao existe um valor universal para o parametro `gamma` que funcione bem em qualquer dataset. Encontrar esse valor requer experimentar diferentes valores. Nesse caso, um valor de `gamma=15` foi capaz de proporcionar um bom resultado.

## Conclusoes:

Nessa aula nos implementamos o PCA com Kernel RBF em Python. Usando o `kernel trick` e uma projecao temporaria em um espaco de maior dimensao, o algoritmo e capaz de comprimir datasets compostos de features nao lineares em um sub-espaco de menor dimensionalidade onde as amostras de classes diferentes sao linearmente separaveis.

# Exercicios

Considerando o seguinte dataset:

In [9]:
from sklearn.datasets import make_circles
X, y = make_circles(100, factor=.1, noise=.1)

plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')

1. Transforme o dataset a seguir usando KPCA. Encontre o valor de `gamma` que maximiza a separabilidade. 
2. Transforme o mesmo dataset para uma menor dimensionalidade utilizando PCA e LDA. Utilize as transformacoes para alimentar um algoritmo de classificacao linear (Perceptron, regressao logistica, etc). Compare os resultaddos com o KPCA otimizado.